## SQL Notebook for Peer Assignment

### Download the datasets

In [1]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql

  Using cached SQLAlchemy-1.3.9-cp310-cp310-macosx_11_0_arm64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.23
    Uninstalling SQLAlchemy-2.0.23:
      Successfully uninstalled SQLAlchemy-2.0.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


### Connect to the database

In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
# !pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [7]:
# Remove blank rows from table
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Tasks

##### Display the names of the unique launch sites  in the space mission

In [8]:
%%sql
select distinct "Launch_Site"
from SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#####  Display 5 records where launch sites begin with the string 'CCA' 

In [9]:
%%sql
select *
from SPACEXTABLE
where "Launch_Site" like "CCA%"
limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [10]:
%%sql
select sum(PAYLOAD_MASS__KG_) as "total payload mass"
from SPACEXTABLE
where Customer = "NASA (CRS)";

 * sqlite:///my_data1.db
Done.


total payload mass
45596


##### Display average payload mass carried by booster version F9 v1.1

In [11]:
%%sql
select avg(PAYLOAD_MASS__KG_) as "average payload mass"
from SPACEXTABLE
where "Booster_Version" = "F9 v1.1";

 * sqlite:///my_data1.db
Done.


average payload mass
2928.4


##### List the date when the first succesful landing outcome in ground pad was acheived.

In [12]:
%%sql 
select min("Date") as "first succesful landing outcome in ground pad"
from SPACEXTABLE
where "Landing_Outcome" = "Success (ground pad)";

 * sqlite:///my_data1.db
Done.


first succesful landing outcome in ground pad
2015-12-22


##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [13]:
%%sql
select distinct "Booster_Version"
from SPACEXTABLE
where "Landing_Outcome" = "Success (drone ship)" and PAYLOAD_MASS__KG_ between 4000 and 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


##### List the total number of successful and failure mission outcomes

In [14]:
%%sql
select "Mission_Outcome", count("Mission_Outcome") as "The Count"
from spacextable
Group by 1;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,The Count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [15]:
%%sql
select distinct "Booster_Version"
from spacextable
where PAYLOAD_MASS__KG_ in (select max(PAYLOAD_MASS__KG_) as "max_payload"
from spacextable)
;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [16]:
%%sql 
select substr(Date, 6,2) as "month", "Landing_Outcome", "Booster_Version", "Launch_Site"
from spacextable
where substr(Date,0,5) = "2015" and "Landing_Outcome" = "Failure (drone ship)"
;

 * sqlite:///my_data1.db
Done.


month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [17]:
%%sql 
select "Landing_Outcome", count("Landing_Outcome") as "the_count"
from spacextable
where date between "2010-06-04" and "2017-03-20"
group by 1
order by 2 desc
;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,the_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
